In [3]:
from scipy.io import arff
import pandas as pd

data = arff.loadarff(r"C:\ECS170\ECS_171_FinalProject\electricity-normalized.arff")
df = pd.DataFrame(data[0])
print(df)

         date   day    period  nswprice  nswdemand  vicprice  vicdemand  \
0      0.0000  b'2'  0.000000  0.056443   0.439155  0.003467   0.422915   
1      0.0000  b'2'  0.021277  0.051699   0.415055  0.003467   0.422915   
2      0.0000  b'2'  0.042553  0.051489   0.385004  0.003467   0.422915   
3      0.0000  b'2'  0.063830  0.045485   0.314639  0.003467   0.422915   
4      0.0000  b'2'  0.085106  0.042482   0.251116  0.003467   0.422915   
...       ...   ...       ...       ...        ...       ...        ...   
45307  0.9158  b'7'  0.914894  0.044224   0.340672  0.003033   0.255049   
45308  0.9158  b'7'  0.936170  0.044884   0.355549  0.003072   0.241326   
45309  0.9158  b'7'  0.957447  0.043593   0.340970  0.002983   0.247799   
45310  0.9158  b'7'  0.978723  0.066651   0.329366  0.004630   0.345417   
45311  0.9158  b'7'  1.000000  0.050679   0.288753  0.003542   0.355256   

       transfer    class  
0      0.414912    b'UP'  
1      0.414912    b'UP'  
2      0.414912   